In [ ]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [ ]:
## load data on 2020 crimes in DC
dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
dc_crim_2020['report_dt'] = pd.to_datetime(dc_crim_2020.REPORT_DAT)

## define crimes to look for and crimes to look within
CCN_examples = ['20165648', '20123250']
crimes_lookfor = dc_crim_2020.loc[dc_crim_2020.CCN.astype(str).isin(CCN_examples),
                ['CCN', 'WARD', 'OFFENSE', 'report_dt']].copy()
other_crimes = dc_crim_2020[~dc_crim_2020.CCN.astype(str).isin(CCN_examples)].copy()

## print crimes_lookfor
crimes_lookfor.head()
# other_crimes.head()

**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [ ]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(0, crimes_lookfor.shape[0]):
    
    ## extract row
    one_row = crimes_lookfor.iloc[i]
    
    ## first, subset to crimes in same ward
    same_wards = other_crimes[other_crimes.WARD == one_row.WARD].copy()
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    cutoff = one_row.report_dt +  timedelta(minutes=1000)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= one_row.report_dt) & 
                                    (same_wards.report_dt <= cutoff)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()


## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

### 2.2 apply it to one of the focal crimes

### 2.3 Use list comprehension to iterate and apply it over the other focal crimes